In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [11]:
import os
for dirname, _, filenames in os.walk('train'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [12]:
acc = pd.read_csv("Accident_Information.csv")
acc.head()

C:\Users\mkvar\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Accident_Index,1st_Road_Class,1st_Road_Number,2nd_Road_Class,2nd_Road_Number,Accident_Severity,Carriageway_Hazards,Date,Day_of_Week,Did_Police_Officer_Attend_Scene_of_Accident,...,Police_Force,Road_Surface_Conditions,Road_Type,Special_Conditions_at_Site,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Year,InScotland
0,200501BS00001,A,3218.0,NaN,0.0,Serious,None,2005-01-04,Tuesday,1.0,...,Metropolitan Police,Wet or damp,Single carriageway,None,30.0,17:42,Urban,Raining no high winds,2005,No
1,200501BS00002,B,450.0,C,0.0,Slight,None,2005-01-05,Wednesday,1.0,...,Metropolitan Police,Dry,Dual carriageway,None,30.0,17:36,Urban,Fine no high winds,2005,No
2,200501BS00003,C,0.0,NaN,0.0,Slight,None,2005-01-06,Thursday,1.0,...,Metropolitan Police,Dry,Single carriageway,None,30.0,00:15,Urban,Fine no high winds,2005,No
3,200501BS00004,A,3220.0,NaN,0.0,Slight,None,2005-01-07,Friday,1.0,...,Metropolitan Police,Dry,Single carriageway,None,30.0,10:35,Urban,Fine no high winds,2005,No
4,200501BS00005,Unclassified,0.0,NaN,0.0,Slight,None,2005-01-10,Monday,1.0,...,Metropolitan Police,Wet or damp,Single carriageway,None,30.0,21:13,Urban,Fine no high winds,2005,No


In [13]:
veh = pd.read_csv("Vehicle_Information.csv", encoding='ISO-8859-1')
acc.shape

(2047256, 34)

In [27]:
df2['Accident_Severity'] = df2['Accident_Severity'].astype('category')

In [28]:
df2 = df2[[c for c in df2 if c not in ['Accident_Severity']] 
       + ['Accident_Severity']]

In [29]:
y = df2['Accident_Severity']
X = df2.iloc[:,:-1]

In [30]:
df2.to_csv("data.csv")

In [31]:
oe = OrdinalEncoder()
oe.fit(X)
X = oe.transform(X)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

# SMOTE

In [34]:
from imblearn.over_sampling import SMOTE


In [35]:
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)

In [36]:
print("Before")
print(y.value_counts())
print("After")
print(pd.Series(y_resampled).value_counts())

Before
Slight     1368445
Serious     199235
Fatal        20633
Name: Accident_Severity, dtype: int64
After
Fatal      1231884
Serious    1231884
Slight     1231884
Name: Accident_Severity, dtype: int64


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=1, stratify = y_resampled)

In [39]:
X_train.shape

(2956521, 15)

# Categorical Naive Bayes

In [40]:
from sklearn.naive_bayes import CategoricalNB
model = CategoricalNB()
model.fit(X_train, y_train)

CategoricalNB()

In [41]:
pred = model.predict(X_test)

In [42]:

from sklearn.metrics import classification_report

In [43]:
report = classification_report(y_test, pred)
print("Naive Bayes(Categorical with SMOTE \n", report)

Naive Bayes(Categorical with SMOTE 
               precision    recall  f1-score   support

       Fatal       0.59      0.70      0.64    246377
     Serious       0.49      0.35      0.41    246377
      Slight       0.61      0.66      0.63    246377

    accuracy                           0.57    739131
   macro avg       0.56      0.57      0.56    739131
weighted avg       0.56      0.57      0.56    739131



# Complement Naive Bayes

In [44]:
from sklearn.naive_bayes import ComplementNB
model2 = ComplementNB()
model2.fit(X_train, y_train)
pred = model2.predict(X_test)

In [45]:
report = classification_report(y_test, pred)
print("\tNaive Bayes(ComplementNB) \n", report)

	Naive Bayes(ComplementNB) 
               precision    recall  f1-score   support

       Fatal       0.44      0.55      0.49    246377
     Serious       0.37      0.48      0.42    246377
      Slight       0.50      0.24      0.33    246377

    accuracy                           0.42    739131
   macro avg       0.44      0.42      0.41    739131
weighted avg       0.44      0.42      0.41    739131



# Bernoulli Naive Bayes

In [46]:
from sklearn.naive_bayes import BernoulliNB
model3 = BernoulliNB()
model3.fit(X_train, y_train)
pred = model3.predict(X_test)

In [47]:
report = classification_report(y_test, pred)
print("\tNaive Bayes(BernouuliNB) \n", report)

	Naive Bayes(BernouuliNB) 
               precision    recall  f1-score   support

       Fatal       0.51      0.65      0.57    246377
     Serious       0.36      0.21      0.26    246377
      Slight       0.48      0.56      0.51    246377

    accuracy                           0.47    739131
   macro avg       0.45      0.47      0.45    739131
weighted avg       0.45      0.47      0.45    739131

